In [201]:
%%bash
rm -rf /home/jordan990301/PCA_Experiments/data/outputs/juicer
mkdir /home/jordan990301/PCA_Experiments/data/outputs/juicer

HIC_PATH="/home/jordan990301/PCA_Experiments/data/Rao2014/samples/GSM1551550_HIC001.hic"
OUTPUT_PATH="/home/jordan990301/PCA_Experiments/data/outputs/juicer"
JUICER_TOOLS_PATH="/home/jordan990301/PCA_Experiments/juicer/juicer_tools.jar"

java -jar $JUICER_TOOLS_PATH pearsons KR $HIC_PATH 6 BP 1000000 $OUTPUT_PATH/pearson_matrix.txt -p
java -jar $JUICER_TOOLS_PATH eigenvector KR $HIC_PATH 6 BP 1000000 $OUTPUT_PATH/eigenvector.txt -p

WARN [2023-11-08T22:27:12,118]  [Globals.java:138] [main]  Development mode is enabled
Reading file: /home/jordan990301/PCA_Experiments/data/Rao2014/samples/GSM1551550_HIC001.hic
WARN [2023-11-08T22:27:12,878]  [Globals.java:138] [main]  Development mode is enabled


In [202]:
import sys
import numpy as np
import pandas as pd

output_path = "/home/jordan990301/PCA_Experiments/data/outputs/juicer"

In [203]:
### Calculate from juicer_tools
pearson_df = pd.read_table(f"{output_path}/pearson_matrix.txt", header=None, sep=" ")
pearson_df.pop(pearson_df.columns[-1])
print(len(pearson_df[0]))
print(len(pearson_df.loc[0]))

172
172


In [204]:
pearson_df = pearson_df.dropna(axis=0, how="all").reset_index(drop=True)
pearson_df = pearson_df.dropna(axis=1, how="all").reset_index(drop=True)

In [205]:
print(pearson_df.isnull().values.any())
print(len(pearson_df[0]))
print(len(pearson_df.loc[0]))

False
170
170


In [206]:
from sklearn.decomposition import PCA
pca = PCA(n_components=len(pearson_df[0]))
pca.fit(pearson_df)

PCA(n_components=170)

In [207]:
# print(pca.explained_variance_) # eigenvalues
print(pca.explained_variance_ratio_[0]) # Percentage of variance explained by each of the selected components.
print(pca.explained_variance_ratio_[1]) # Percentage of variance explained by each of the selected components.
print(pca.explained_variance_ratio_[2]) # Percentage of variance explained by each of the selected components.

0.9120964808266161
0.04210297149608417
0.026140316241396643


In [208]:
My_PC1 = pca.components_[0]
print(len(My_PC1))
print(type(My_PC1)) ### Make sure it is Numpy format

np.savetxt(f'{output_path}/My_PC1.txt', My_PC1, delimiter='\n', fmt='%1.4f')

170
<class 'numpy.ndarray'>


In [209]:
Juicer_PC1 = pd.read_table(f"{output_path}/eigenvector.txt", header=None, sep=" ")
Juicer_PC1 = Juicer_PC1.dropna(axis=0, how="all").reset_index(drop=True)
Juicer_PC1 = Juicer_PC1[0]
Juicer_PC1 = Juicer_PC1.to_numpy()
print(len(Juicer_PC1))
print(type(Juicer_PC1)) ### Make sure it is Numpy format

np.savetxt(f'{output_path}/Juicer_PC1.txt', Juicer_PC1, delimiter='\n', fmt='%1.4f')

170
<class 'numpy.ndarray'>


In [210]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(My_PC1, Juicer_PC1) / (norm(My_PC1) * norm(Juicer_PC1))
cos_sim

-0.9839977437568964

In [211]:
My_PC1_Pos = My_PC1 > 0
Juicer_PC1_Pos = Juicer_PC1 > 0

PC1_Difference_Count = 0
for i in range(My_PC1_Pos.shape[0]):
    # print(i, My_PC1_Pos[i])
    if(My_PC1_Pos[i] != Juicer_PC1_Pos[i]):
        PC1_Difference_Count += 1

print(PC1_Difference_Count)

167
